In [188]:
#Importando Bibliotecas
import pandas as pd
from datetime import timedelta
import scipy.stats as stats

In [258]:
#Importando as Bases
df_full = pd.read_csv('5.SERV_FALEC_EXON.csv', sep=';', encoding='UTF-8').assign(IDADE = '')

df_full['DT_NASC_SERVIDOR'] = pd.to_datetime(df_full['DT_NASC_SERVIDOR'], yearfirst=True, format='%d/%m/%Y')
df_full['DT_BASE'] = '31/12/2020'
df_full['DT_BASE'] = pd.to_datetime(df_full.DT_BASE, yearfirst=True, format='%d/%m/%Y')
df_full['IDADE'] = round((df_full.DT_BASE - df_full.DT_NASC_SERVIDOR) / timedelta(days=365.25) - 0.5).astype(int)

In [193]:
#Importando as Tábuas
df_tabuas = pd.read_excel('Funções Biométricas (Com Fórmulas).xlsx', sheet_name='qx')

In [227]:
ary_expostos = []
ary_esperados = []
for l in range(126+1):
    linha = []
    linha2 = []
    #Criando uma matriz com a quantidade de registros de cada IDADE
    for c in range(1):
        qtda = len(df_full[df_full['IDADE'] == l]) #Quantidade de eventos ocorridos em cada idade
        linha.append(qtda)
    ary_expostos.append(linha)

    #Criando uma matriz com o número de MORTES esperadas por idade e o X² calculado
    for c2 in range(len(df_tabuas.columns)):
        esperado = ary_expostos[l][0] * df_tabuas.iloc[l, c2] #Expostos * qx da tábua
        linha2.append(esperado)
    ary_esperados.append(linha2)

df_expostos = pd.DataFrame(data=ary_expostos)
df_esperados = pd.DataFrame(data=ary_esperados, columns=df_tabuas.columns.values)

In [256]:
#Cálculo do X²
ary_qui_quadrado = []
for l3 in range(127):
    linha3 = []
    for c3 in range(len(df_tabuas.columns)):
        if df_esperados.iloc[l3, c3] == 0.0:
            qui_quadrado = 0
            linha3.append(qui_quadrado)
        elif df_esperados.iloc[l3, c3] != 0.0:
            qui_quadrado = (ary_expostos[l3][0] - df_esperados.iloc[l3, c3])**2 / df_esperados.iloc[l3, c3]
            linha3.append(qui_quadrado)
    ary_qui_quadrado.append(linha3)

df_qui_quadrado = pd.DataFrame(data=ary_qui_quadrado, columns=df_tabuas.columns.values)
df_qui_quadrado.loc['qui_quadrado'] = round(df_qui_quadrado.sum(axis=0), 2)
df_qui_quadrado.loc['p_value'] = 1-stats.chi2.cdf(x=df_qui_quadrado.loc['qui_quadrado'], df = int(df_expostos[df_expostos != 0].count() - 1))
qui_quadrado_tabu = stats.chi2.ppf(q = 0.95, df = int(df_expostos[df_expostos != 0].count() - 1))

In [271]:
#Cálculo do Kolmogorov-Smirnov K_S

df_esperados_acum = df_esperados.cumsum()
df_expostos_acum = df_expostos.cumsum()
ary_k_s = []
for l in range(126+1):
    linha2 = []
    for c2 in range(len(df_tabuas.columns)):
        k_s_calculado =  round(abs(((float(df_expostos_acum.loc[l]) / int(df_expostos.sum()))) - (float(df_esperados_acum.iloc[l, c2]) / df_esperados.iloc[:,c2].sum())), 6)
        linha2.append(k_s_calculado)
    ary_k_s.append(linha2)

df_k_s = pd.DataFrame(data=ary_k_s, columns=df_tabuas.columns.values)
df_k_s.loc['k_s'] = df_k_s.max(axis=0)
df_k_s.loc['p_value'] = stats.chi2.cdf(x=df_k_s.loc['k_s'], df = int(df_expostos[df_expostos != 0].count()))
df_k_s.tail()

,ALLG-72,AMERICAN EXPERIENCE,AT2000 (Suavizada 10%)_FEM,AT2000 (Suavizada 10%)_MAS,AT-2000 FEMALE,AT-2000 MALE,AT-49 FEMALE,AT-49 MALE,AT-50,AT-55,...,RRB-1944 Mod - Masc,TASA-1927,WYATT 1985,ZIMMERMANN EMPR. ESCRIT,Atuários Ingleses,Hubbard Laffitte,J.P. Jansen,Kinkelin,Moser,Samuel Dumas
124,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
125,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
126,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
k_s,0.0,2.931700e-01,4.809130e-01,4.766320e-01,4.811570e-01,4.764090e-01,4.814830e-01,5.033970e-01,5.079580e-01,5.005000e-01,...,4.755310e-01,8.874280e-01,8.463710e-01,6.294760e-01,1.603110e-01,1.529530e-01,1.958860e-01,1.926430e-01,2.119030e-01,1.534380e-01
p_value,0.0,2.432499e-51,1.413575e-45,1.112668e-45,1.432900e-45,1.098815e-45,1.459115e-45,4.801791e-45,6.112409e-45,4.114652e-45,...,1.045871e-45,1.774323e-38,5.036995e-39,1.887398e-42,2.165962e-58,6.112906e-59,4.767433e-56,3.042350e-56,3.949492e-55,6.656857e-59


In [274]:
df_expostos.sum()

0    783
dtype: int64

In [273]:
#Cálculo do Erro Quadrático Médio
ary_DQM = []
for l4 in range(127):
    linha4 = []
    for c4 in range(len(df_tabuas.columns)):
        if df_esperados.iloc[l4, c4] == 0.0:
            DQM = 0
            linha4.append(DQM)
        elif df_esperados.iloc[l4, c4] != 0.0:
            DQM = (ary_expostos[l4][0] - df_esperados.iloc[l4, c4])**2 / ary_expostos[l4][0]
            linha4.append(DQM)
    ary_DQM.append(linha4)

df_DQM = pd.DataFrame(data=ary_DQM, columns=df_tabuas.columns.values)
df_DQM.loc['DQM'] = round(df_DQM.sum(axis=0), 2)
df_DQM

,ALLG-72,AMERICAN EXPERIENCE,AT2000 (Suavizada 10%)_FEM,AT2000 (Suavizada 10%)_MAS,AT-2000 FEMALE,AT-2000 MALE,AT-49 FEMALE,AT-49 MALE,AT-50,AT-55,...,RRB-1944 Mod - Masc,TASA-1927,WYATT 1985,ZIMMERMANN EMPR. ESCRIT,Atuários Ingleses,Hubbard Laffitte,J.P. Jansen,Kinkelin,Moser,Samuel Dumas
0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.000000e+00,0.000000e+00,0.00,0.00,0.00,0.00
1,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.000000e+00,0.000000e+00,0.00,0.00,0.00,0.00
2,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.000000e+00,0.000000e+00,0.00,0.00,0.00,0.00
3,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.000000e+00,0.000000e+00,0.00,0.00,0.00,0.00
4,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.000000e+00,0.000000e+00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.000000e+00,0.000000e+00,0.00,0.00,0.00,0.00
124,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.000000e+00,0.000000e+00,0.00,0.00,0.00,0.00
125,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.000000e+00,0.000000e+00,0.00,0.00,0.00,0.00
126,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.000000e+00,0.000000e+00,0.00,0.00,0.00,0.00


In [184]:
df_result = pd.DataFrame()

df_result['X²'] = df_qui_quadrado.loc['qui_quadrado']
df_result['[X²] p-valor'] = df_qui_quadrado.loc['p_value']
df_result['K-S'] = df_k_s.loc['k_s']
df_result['[K-S] p-valor'] = df_k_s.loc['p_value']
df_result['DQM'] = df_DQM.loc['DQM']

df_result

,X²,[X²] p-valor,K-S,[K-S] p-valor,DQM
ALLG-72,54.21,0.428054,0.000000,0.000000e+00,7.047000e+01
AMERICAN EXPERIENCE,66848.56,0.000000,0.293170,2.432499e-51,7.580800e+02
AT2000 (Suavizada 10%)_FEM,1169660.67,0.000000,0.480913,1.413575e-45,7.801200e+02
AT2000 (Suavizada 10%)_MAS,705222.48,0.000000,0.476632,1.112668e-45,7.781400e+02
AT-2000 FEMALE,1049534.17,0.000000,0.481157,1.432900e-45,7.797900e+02
...,...,...,...,...,...
Hubbard Laffitte,4944895.16,0.000000,0.152953,6.112906e-59,4.335079e+10
J.P. Jansen,5183.02,0.000000,0.195886,4.767433e-56,6.103133e+04
Kinkelin,5287.92,0.000000,0.192643,3.042350e-56,5.835048e+04
Moser,5878.99,0.000000,0.211903,3.949492e-55,7.418632e+04


In [186]:
#df_result.to_excel('Quadro Testes.xlsx',)

In [212]:
#tupla = (1, 5 ,9)
#tupla[0] = 2
#tupla

#lista = [[1, 2], 5 ,9]
#lista[0][0]
#lista

dicio = {'cor': [1, 2], 'carro': ['uno', 'gol']}
dicio

{'cor': [1, 2], 'carro': ['uno', 'gol']}